In [4]:
from scrapping import *
from fonctions_preprocess import *

In [5]:
B = pd.read_csv("/home/onyxia/work/Projet-python/Fbref_alex/SOCCER_241223.csv", index_col=0)
C = pd.read_csv("/home/onyxia/work/Projet-python/Fbref_alex/recup_matchweek.csv", index_col=0)

In [6]:
mapping_equipe = {
    'Nimes': 'Nîmes',
    'Paris S-G': 'Paris Saint Germain',
    'Saint Etienne': 'Saint-Étienne'
}
pd.set_option('display.max_columns', None)

In [7]:
from datetime import datetime

ajd = datetime.now()

B = preprocess_initial(B, mapping_equipe=mapping_equipe)
C = preprocess_initial(C, mapping_equipe)
B = renommer_colonnes(B)
C = renommer_colonnes(C)
B = columns_to_keep(B)
C = columns_to_keep(C)
B = preprocess_variables(B)
resultat = pd.concat([C, B], sort=False).reset_index(drop= True)
resultat = preparation_model(resultat)
resultat = preprocess_data(resultat)


In [9]:
modelisation(resultat, ajd).to_csv("futur_result.csv")

/home/onyxia/work/Projet-python/Fbref_alex/TestAutom/fonctions_preprocess.py:442: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  y_train = X_train[df['DateTime'] <= cutoff_date]["Result"]


In [ ]:
A = pd.read_csv("result.csv", index_col=0)
A["DateTime"] = pd.to_datetime(A["DateTime"])
derniere_date = A["DateTime"].max()

if derniere_date <= ajd:

    base_actuelle = pd.read_csv("/home/onyxia/work/Projet-python/Fbref_alex/SOCCER_241223.csv", index_col=0)
    data = scrape_latest_ligue1_data()
    base_supplémentaire = data[0]
    base_updated = add_new_matches(base_initiale=base_actuelle, base_nouvelle=base_supplémentaire)
    base_updated.to_csv("/home/onyxia/work/Projet-python/Fbref_alex/SOCCER_241223.csv")

    recup_matchweek = data[1]

    recup_matchweek=find_futur_matchweeks(recup_matchweek, mapping_equipe)

    if recup_matchweek == None:
        print("Pas de nouveaux matchs à récupérer")
    else:
        
        recup_matchweek2 = recup_matchweek.copy()
        recup_matchweek2 = preprocess_initial(recup_matchweek2, mapping_equipe)
        recup_matchweek2 = renommer_colonnes(recup_matchweek2)
        recup_matchweek2 = columns_to_keep(recup_matchweek2)

        base_updated2 = base_updated.copy()
        base_updated2 = preprocess_initial(base_updated2, mapping_equipe)
        base_updated2 = renommer_colonnes(base_updated2)
        base_updated2 = columns_to_keep(base_updated2)
        base_updated2 = preprocess_variables(base_updated2)

        resultat = pd.concat([base_updated2, recup_matchweek2], sort=False).reset_index(drop= True)
        resultat = preparation_model(resultat)
        resultat = preprocess_data(resultat, ajd)


        A = pd.concat([A, resultat]).sort_values(by="DateTime").reset_index(drop= True)
        A_updated = A.merge(df[['MatchID', 'Result']], on='MatchID', how='left')
        A.to_csv("futur_result.csv")
        

        


    #base_updated.to_csv("/home/onyxia/work/Projet-python/Fbref_alex/SOCCER_241223.csv")
    #base_updated.to_csv("Fbref_alex/SOCCER_241223.csv")
    


    #recup_matchweek.to_csv("/home/onyxia/work/Projet-python/Fbref_alex/recup_matchweek.csv")
    #recup_matchweek.to_csv("Fbref_alex/recup_matchweek.csv")
    